In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter, welch

In [2]:
# From the poisson train generation notebook

def inhomogeneous_poisson_through_num_points_for_window_one(lambdas):
    t = np.zeros(len(lambdas))
    for i, lambd in enumerate(lambdas):
        num_points = np.random.poisson(lambd)
        if num_points > 0: t[i] = 1
    return t

def generate_pink_noise(num_obs, mean = 1, std = 0.5):

    white_noise = np.random.normal(mean, std, num_obs + 2000)

    A = [1, -2.494956002, 2.017265875, -0.522189400]
    B = [0.049922035, -0.095993537, 0.050612699, -0.004408786]

    pink_noise = lfilter(B, A, white_noise)[2000:]

    return pink_noise

In [3]:
def generate_train_matrix(N_syn, T):
    matrix = np.zeros((N_syn, T))
    for i in range(N_syn):
        matrix[i, :] = inhomogeneous_poisson_through_num_points_for_window_one(generate_pink_noise(num_obs = T, mean = 2) / 1000)
    return matrix

def shift_wrap_array(arr, by):
    wrap = arr[-by:].copy()
    arr[by:] = arr[0:-by]
    arr[0:by] = wrap
    return arr

def compute_frs_from_probs(spike_probs):
    '''in units mHz'''
    return -np.log(1 - spike_probs)

def shift_mean(frs, desired_mean):
    '''frs: np.array of firing rates that will be lamda
    units mHz'''
    frs = frs + (desired_mean - np.mean(frs))
    return frs

def generate_delayed_fr_profile(exc_trains_matrix, desired_mean):
    '''trains_matrix is n_pre_cells x n_timepoints'''
    # all of the following are 1x n _timepoints
    exc_spike_probs = np.mean(exc_trains_matrix, axis = 0) # merge all exc spike trains
    inh_spike_probs = shift_wrap_array(exc_spike_probs, 4) # delay the exc spike train
    inh_frs = compute_frs_from_probs(inh_spike_probs)
    inh_frs = shift_mean(inh_frs, desired_mean=desired_mean)
    return inh_frs

In [4]:
def generate_delayed_fr_profile(exc_trains_matrix, desired_mean):
    '''trains_matrix is n_pre_cells x n_timepoints'''
    # all of the following are 1x n _timepoints
    exc_spike_probs = np.mean(exc_trains_matrix, axis = 0) # merge all exc spike trains
    inh_spike_probs = shift_wrap_array(exc_spike_probs, 4) # delay the exc spike train
    inh_frs = -np.log(1 - inh_spike_probs)
    inh_frs = shift_mean(inh_frs, desired_mean=desired_mean)
    return inh_frs

In [5]:
# Generate a matrix of excitatory trains (we will use uniform lambdas, but we can also use 1/f or anything else)
np.random.seed(123)
N_syn = 10
T = 1000 # 5 seconds

matrix = generate_train_matrix(N_syn, T)

In [6]:
[1,10,100]

[1, 10, 100]

In [7]:
type(matrix)

numpy.ndarray

In [8]:
matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0.

In [9]:
matrix = [[   175    796   1455   1585   1755   2061   2129   2745   2876   2966
   3127   3187   3683   3762   3966   4086   5571   6123   6830   7101
   7148   7282   7285   7372   7381   7384   7572   8008   8777   8858
   9208   9887  10889  11180  11283  11642  12220  12250  12358  12521
  12541  12644  12650  12766  12833  13098  13105  13383  13410  13630
  13660  13807  14011  14807  14977  14991  15155  15184  15457  15827
  15858  16756  16796  16968  17062  17265  17289  17318  17354  17412
  17514  17536  18242  18358  18731  18782  18977  19392  19627  19896
  20052  20262  20277  20709  20770  20806  21129  21522  21945  22164
  22703  22858  23052  23261  23331  24515  24691  24714  25418  25698
  26020  26425  27409  27430  27905  28156  29031  29284  30285  30325
  30560  30812  31006  31378  31646  31680  31708  32114  32137  32254
  32348  32488  32807  32834  33341  33413  33679  33803  33814  33931
  34521  34658  35009  35273  35380  35523  35776  35843  35900  36142
  36145  36239  36291  36662  36855  37016  37269  37432  37467  37723
  37732  37897  37901  38009  38312  39573  39789  40024  40368  40401
  40509  40527  40884  40962  41290  41833  43050  43051  43674  44025
  44177  45058  45059  45278  45339  45408  45425  45637  45659  45892
  46595  46606  46943  47410  48527  48630  48850  48854  48922  49331
  49504  49972  50255  50557  50959  51302  52230  52347  52497  52854
  53287  53378  53777  54088  54536  54844  55131  55522  55548  55563
  56290  56318  56817  56833  56867  57389  57447  57559  57866  57925
  57946  58143  58271  58547  58626  58716  59078  60138  60301  60730
  61007  61301  61415  61455  61952  62094  62364  62451  62593  63094
  63473  63651  63776  63859  64709  64769  64811  65139  65538  65923
  66068  66163  66167  66380  66767  67329  67575  67585  67892  68009
  68781  68832  69527  69539  69891  69996  70059  70181  70208  70245
  70860  70886  71410  71440  71457  71555  71562  71575  71837  72246
  72765  72804  73062  73390  73442  73671  73809  73830  73885  74592
  75176  75309  75704  75966  76574  76678  77383  77427  77978  78080
  78568  79434  79591  79943  80566  80708  80965  81155  81387  81491
  81562  82377  82648  83064  83192  83202  83312  83424  83599  83661
  84199  84481  84928  84958  85504  86379  86465  86847  86907  87157
  87585  87615  87840  88961  89264  89336  89347  89705  89979  90003
  90205  90390  90405  90808  91267  91351  91445  91553  91554  91711
  91955  92243  92394  92508  93305  93561  93888  94597  94940  95180
  95184  95239  95306  95475  96021  96400  96671  96935  96968  97067
  97565  97692  97988  98039  98314  98434  98947  99774 100348 100949
 100982 101994 102660 103452 104088 104298 104372 104858 105240 105401
 105649 105673 105822 105985 106333 106573 106912 107042 107406 107560
 108121 108270 108538 108901 109577 109628 109721 110081 110182 110233
 110339 110622 110728 111021 111120 111352 111960 111990 112347 112553
 112801 112930 112947 113262 113276 113894 113941 113996 114113 114740
 114975 115605 115768 115868 115880 115936 116009 116449 116682 117108
 117543 117851 118216 118645 118785 119056 119135 119524 119885 119894]...]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4129026192.py, line 1)

In [ ]:
matrix.shape

In [ ]:
# Compute the array of probabilities of observing at least one spike at time point t
exc_spike_prob = np.mean(matrix, axis = 0)
exc_spike_prob[:10]

In [ ]:
exc_spike_prob.shape

In [ ]:
# Shift the array to get probabilities for inh synapses
inh_spike_prob = shift_wrap_array(exc_spike_prob, 4)
inh_spike_prob[:10]

To convert from probabilities to firing rates, we can use the following analysis.

$$
p = p(N_{sp} > 0) = 1 - p(N_{sp} = 0) = 1 - \frac{e^{-\lambda}\lambda^0}{0!} = 1 - e^{-\lambda}.
$$

Then

$$
\begin{align*}
e^{-\lambda} &= 1 - p \\
\lambda &= -\log(1-p)
\end{align*}
$$

In [ ]:
exc_fr = -np.log(1 - exc_spike_prob)
inh_fr = -np.log(1 - inh_spike_prob)

# Check that the mean rates are preserved at this point
print("Exc MFR from probabilities: ", np.mean(exc_fr))
print("Inh MFR from probabilities: ", np.mean(inh_fr))

In [ ]:
def compute_frs_from_probs(spike_probs):
    '''in units mHz'''
    return -np.log(1 - spike_probs)

In [ ]:
exc_fr

In [ ]:
inh_spikes

In [ ]:
np.mean(inh_fr)
new_fr = shift_mean(inh_fr, desired_mean=0.008)
print(np.mean(new_fr))

In [ ]:
def shift_mean(frs, desired_mean):
    '''frs: np.array of firing rates that will be lamda
    units mHz'''
    frs = frs + (desired_mean - np.mean(frs))
    return frs

np.random.seed(123)
exc_spikes = inhomogeneous_poisson_through_num_points_for_window_one(exc_fr)
inh_spikes = inhomogeneous_poisson_through_num_points_for_window_one(shift_mean(inh_fr, desired_mean=0.008)) # 4.5 = desired inh mean firing rate

fig, ax = plt.subplots(1, 2, figsize = (20, 5))
ax[0].stem(exc_spikes, markerfmt = ' ')
ax[1].stem(inh_spikes, markerfmt = ' ', linefmt = '--')

ax[0].set_title("Average Excitatory Train")
ax[1].set_title("Inhibitory Train")

print(f"FR exc spikes: {np.mean(exc_spikes) * 1000 } Hz")
print(f"FR inh spikes: {np.mean(inh_spikes) * 1000} Hz")

In [ ]:
exc

In [ ]:
plt.plot(range(len(exc_spike_prob)), exc_spike_prob*N_syn)